## Persona

👤 Persona : Léna, écologue spécialisée en biodiversité antarctique
Profil :
Âge : 34 ans

Fonction : Chargée de recherche dans un institut de conservation de la faune sauvage.

Outils utilisés : Python, Databricks, R, QGIS

Objectif : Étudier l’évolution morphologique des manchots (penguins) en Antarctique selon les espèces, les îles d’observation et les années, dans un contexte de changement climatique.

Besoins :

- Visualiser les différences morphologiques selon les espèces (Adélie, Gentoo, Chinstrap).

- Étudier les relations entre les caractéristiques physiques : taille du bec, nageoires, poids.

- Comparer les individus selon leur sexe et leur localisation (île).

- Observer l’évolution interannuelle (par année) des mesures.

- Détecter des tendances ou des anomalies morphologiques liées au réchauffement ou à la nutrition.

📊 Indicateurs pertinents pour Léna :

- Poids moyen par espèce et par sexe

- Longueur moyenne du bec selon l’île

- Comparaison graphique des nageoires par espèce

- Corrélation entre bill_length_mm et body_mass_g

- Évolution annuelle de la morphologie moyenne par espèce

- Répartition des individus par sexe et île

## Préparation des données

In [2]:
case class Penguin(
  species: String,
  island: String,
  bill_length_mm: Option[Double],
  bill_depth_mm: Option[Double],
  flipper_length_mm: Option[Int],
  body_mass_g: Option[Int],
  sex: String,
  year: Int
)


defined class Penguin


## Récupération des données

In [3]:

val data = sc.textFile("../csv/penguins.csv")
  .mapPartitionsWithIndex { (idx, iter) =>
    val lines = if (idx == 0) iter.drop(1) else iter
    lines.map { line =>
      val cols = line.split(",", -1).map(_.trim)
      Penguin(
        species = cols(1),
        island = cols(2),
        bill_length_mm = scala.util.Try(cols(3).toDouble).toOption,
        bill_depth_mm = scala.util.Try(cols(4).toDouble).toOption,
        flipper_length_mm = scala.util.Try(cols(5).toInt).toOption,
        body_mass_g = scala.util.Try(cols(6).toInt).toOption,
        sex = cols(7),
        year = scala.util.Try(cols(8).toInt).getOrElse(0)
      )
    }
  }.toDF()

  data.show(100)

+-------+---------+--------------+-------------+-----------------+-----------+------+----+
|species|   island|bill_length_mm|bill_depth_mm|flipper_length_mm|body_mass_g|   sex|year|
+-------+---------+--------------+-------------+-----------------+-----------+------+----+
| Adelie|Torgersen|          39.1|         18.7|              181|       3750|  male|2007|
| Adelie|Torgersen|          39.5|         17.4|              186|       3800|female|2007|
| Adelie|Torgersen|          40.3|         18.0|              195|       3250|female|2007|
| Adelie|Torgersen|          NULL|         NULL|             NULL|       NULL|      |2007|
| Adelie|Torgersen|          36.7|         19.3|              193|       3450|female|2007|
| Adelie|Torgersen|          39.3|         20.6|              190|       3650|  male|2007|
| Adelie|Torgersen|          38.9|         17.8|              181|       3625|female|2007|
| Adelie|Torgersen|          39.2|         19.6|              195|       4675|  male|2007|

data = [species: string, island: string ... 6 more fields]


[species: string, island: string ... 6 more fields]

## Atelier 2 / Étape 1 - RDD vers un dataframe

In [4]:
data.printSchema()
println(s"nb de lignes : ${data.count()}")
data.summary().show()

println("Espèces étudiés : ")
data.select("species").distinct().show()

println("Statistiques sur la longueur du bec :")
data.select("bill_length_mm").filter($"bill_length_mm".isNotNull).summary().show()


root
 |-- species: string (nullable = true)
 |-- island: string (nullable = true)
 |-- bill_length_mm: double (nullable = true)
 |-- bill_depth_mm: double (nullable = true)
 |-- flipper_length_mm: integer (nullable = true)
 |-- body_mass_g: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- year: integer (nullable = false)

nb de lignes : 344
+-------+-------+---------+-----------------+------------------+------------------+-----------------+----+------------------+
|summary|species|   island|   bill_length_mm|     bill_depth_mm| flipper_length_mm|      body_mass_g| sex|              year|
+-------+-------+---------+-----------------+------------------+------------------+-----------------+----+------------------+
|  count|    344|      344|              342|               342|               342|              342| 344|               344|
|   mean|   NULL|     NULL| 43.9219298245614| 17.15116959064327|200.91520467836258|4201.754385964912|NULL|2008.0290697674418|
| stddev| 

## Atelier 2 / Étape 2 - Extraction de dimensions

In [5]:
import org.apache.spark.sql.functions.monotonically_increasing_id

val dfSpecies = data
  .select("species")
  .distinct()
  .withColumn("id_species", monotonically_increasing_id())

val dfYears = data
  .select("year")
  .distinct()
  .withColumn("id_year", monotonically_increasing_id())
  .orderBy("year")

val dfIslands = data
  .select("island")
  .distinct()
  .withColumn("id_island", monotonically_increasing_id())

val dfSex = data
  .select("sex")
  .filter($"sex".isNotNull && $"sex" =!= "")
  .distinct()
  .withColumn("id_sex", monotonically_increasing_id())

val factTable = data
  .join(dfSpecies, Seq("species"), "left")
  .join(dfIslands, Seq("island"), "left")  
  .join(dfYears, Seq("year"), "left")
  .join(dfSex, Seq("sex"), "left")
  .select(
    $"id_species",
    $"id_island", 
    $"id_year",
    $"id_sex",
    $"bill_length_mm",
    $"bill_depth_mm",
    $"flipper_length_mm",
    $"body_mass_g"
  )

factTable.show(20)

+----------+---------+-------+------+--------------+-------------+-----------------+-----------+
|id_species|id_island|id_year|id_sex|bill_length_mm|bill_depth_mm|flipper_length_mm|body_mass_g|
+----------+---------+-------+------+--------------+-------------+-----------------+-----------+
|         1|        1|      0|     1|          39.1|         18.7|              181|       3750|
|         1|        1|      0|     0|          39.5|         17.4|              186|       3800|
|         1|        1|      0|     0|          40.3|         18.0|              195|       3250|
|         1|        1|      0|  NULL|          NULL|         NULL|             NULL|       NULL|
|         1|        1|      0|     0|          36.7|         19.3|              193|       3450|
|         1|        1|      0|     1|          39.3|         20.6|              190|       3650|
|         1|        1|      0|     0|          38.9|         17.8|              181|       3625|
|         1|        1|      0|

dfSpecies = [species: string, id_species: bigint]
dfYears = [year: int, id_year: bigint]
dfIslands = [island: string, id_island: bigint]
dfSex = [sex: string, id_sex: bigint]
factTable = [id_species: bigint, id_island: bigint ... 6 more fields]


[id_species: bigint, id_island: bigint ... 6 more fields]

## Atelier 2 / Étape 3 - Tables Hive, SQL

In [6]:
dfSpecies.write.mode("overwrite").saveAsTable("dim_species")
dfIslands.write.mode("overwrite").saveAsTable("dim_islands") 
dfYears.write.mode("overwrite").saveAsTable("dim_years")
dfSex.write.mode("overwrite").saveAsTable("dim_sex")

factTable.write.mode("overwrite").saveAsTable("fact_penguins")

import org.apache.spark.sql.hive.HiveContext
val hc = new HiveContext(sc)

hc.sql("SELECT * FROM dim_species").show()
hc.sql("SELECT COUNT(*) as nb_especes FROM dim_species").show()
hc.sql("SELECT f.* FROM fact_penguins f JOIN dim_species s ON f.id_species = s.id_species").show(10)

+---------+----------+
|  species|id_species|
+---------+----------+
|   Gentoo|         0|
|   Adelie|         1|
|Chinstrap|         2|
+---------+----------+

+----------+
|nb_especes|
+----------+
|         3|
+----------+

+----------+---------+-------+------+--------------+-------------+-----------------+-----------+
|id_species|id_island|id_year|id_sex|bill_length_mm|bill_depth_mm|flipper_length_mm|body_mass_g|
+----------+---------+-------+------+--------------+-------------+-----------------+-----------+
|         0|        2|      0|     0|          46.1|         13.2|              211|       4500|
|         0|        2|      0|     1|          50.0|         16.3|              230|       5700|
|         0|        2|      0|     0|          48.7|         14.1|              210|       4450|
|         0|        2|      0|     1|          50.0|         15.2|              218|       5700|
|         0|        2|      0|     1|          47.6|         14.5|              215|       54

hc = org.apache.spark.sql.hive.HiveContext@1d14f01b


org.apache.spark.sql.hive.HiveContext@1d14f01b

## Atelier 3 - Spark SQL (suite), Stats

In [ ]:
println("Statistiques du poids par espèce et sexe : ")
val poidsParEspeceEtSexe = hc.sql("""
  SELECT s.species, sex.sex, 
         AVG(f.body_mass_g) as poids_moyen,
         COUNT(*) as nb_individus
  FROM fact_penguins f
  JOIN dim_species s ON f.id_species = s.id_species
  JOIN dim_sex sex ON f.id_sex = sex.id_sex
  WHERE f.body_mass_g IS NOT NULL
  GROUP BY s.species, sex.sex
  ORDER BY s.species, sex.sex
""")
poidsParEspeceEtSexe.show()

println("Statistiques de la longueur du bec par ile : ")
val becParIle = hc.sql("""
  SELECT i.island, 
         AVG(f.bill_length_mm) as longueur_bec_moyenne,
         AVG(f.bill_depth_mm) as profondeur_bec_moyenne,
         COUNT(*) as nb_mesures
  FROM fact_penguins f
  JOIN dim_islands i ON f.id_island = i.id_island
  WHERE f.bill_length_mm IS NOT NULL AND f.bill_depth_mm IS NOT NULL
  GROUP BY i.island
  ORDER BY longueur_bec_moyenne DESC
""")
becParIle.show()

println("Comparaison des nageoires par espèce : ")
val nageoiresParEspece = hc.sql("""
  SELECT s.species,
         AVG(f.flipper_length_mm) as nageoire_moyenne,
         MIN(f.flipper_length_mm) as nageoire_min,
         MAX(f.flipper_length_mm) as nageoire_max,
         STDDEV(f.flipper_length_mm) as ecart_type,
         COUNT(*) as nb_mesures
  FROM fact_penguins f
  JOIN dim_species s ON f.id_species = s.id_species
  WHERE f.flipper_length_mm IS NOT NULL
  GROUP BY s.species
  ORDER BY nageoire_moyenne DESC
""")
nageoiresParEspece.show()

println("Corrélation entre longueur du bec et poids par espèce : ")
val correlationParEspece = hc.sql("""
  SELECT s.species,
         CORR(f.bill_length_mm, f.body_mass_g) as correlation_bec_poids,
         COUNT(*) as nb_observations,
         AVG(f.bill_length_mm) as bec_moyen,
         AVG(f.body_mass_g) as poids_moyen
  FROM fact_penguins f
  JOIN dim_species s ON f.id_species = s.id_species
  WHERE f.bill_length_mm IS NOT NULL AND f.body_mass_g IS NOT NULL
  GROUP BY s.species
  ORDER BY correlation_bec_poids DESC
""")
correlationParEspece.show()

println("Évolution annuelle de la morphologie moyenne (poids, bec, nageoire) par espèce : ")
val evolutionAnnuelle = hc.sql("""
  SELECT s.species, y.year,
         AVG(f.body_mass_g) as poids_moyen,
         AVG(f.bill_length_mm) as bec_moyen,
         AVG(f.flipper_length_mm) as nageoire_moyenne,
         COUNT(*) as nb_individus
  FROM fact_penguins f
  JOIN dim_species s ON f.id_species = s.id_species
  JOIN dim_years y ON f.id_year = y.id_year
  WHERE f.body_mass_g IS NOT NULL
  GROUP BY s.species, y.year
  ORDER BY s.species, y.year
""")
evolutionAnnuelle.show()

println("Répartition des individus par sexe et île : ")
val repartitionSexeIle = hc.sql("""
  SELECT i.island, sex.sex, s.species,
         COUNT(*) as nb_individus,
         ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY i.island), 2) as pourcentage
  FROM fact_penguins f
  JOIN dim_islands i ON f.id_island = i.id_island
  JOIN dim_sex sex ON f.id_sex = sex.id_sex
  JOIN dim_species s ON f.id_species = s.id_species
  GROUP BY i.island, sex.sex, s.species
  ORDER BY i.island, s.species, sex.sex
""")
repartitionSexeIle.show()


Statistiques du poids par espèce et sexe : 
+---------+------+------------------+------------+
|  species|   sex|       poids_moyen|nb_individus|
+---------+------+------------------+------------+
|   Adelie|female|3368.8356164383563|          73|
|   Adelie|  male|4043.4931506849316|          73|
|Chinstrap|female| 3527.205882352941|          34|
|Chinstrap|  male| 3938.970588235294|          34|
|   Gentoo|female| 4679.741379310345|          58|
|   Gentoo|  male| 5484.836065573771|          61|
+---------+------+------------------+------------+

Statistiques de la longueur du bec par ile : 
+---------+--------------------+----------------------+----------+
|   island|longueur_bec_moyenne|profondeur_bec_moyenne|nb_mesures|
+---------+--------------------+----------------------+----------+
|   Biscoe|  45.257485029940106|    15.874850299401194|       167|
|    Dream|   44.16774193548386|    18.344354838709677|       124|
|Torgersen|  38.950980392156865|    18.429411764705883|        5

poidsParEspeceEtSexe = [species: string, sex: string ... 2 more fields]
becParIle = [island: string, longueur_bec_moyenne: double ... 2 more fields]
nageoiresParEspece = [species: string, nageoire_moyenne: double ... 4 more fields]
correlationParEspece = [species: string, correlation_bec_poids: double ... 3 more fields]
evolutionAnnuelle = [species: string, year: int ... 4 more fields]
repartitionSexeIle = [island: string, sex: string ... 3 more fields]


[island: string, sex: string ... 3 more fields]

In [21]:
poidsParEspeceEtSexe.coalesce(1)
  .write
  .mode("overwrite")
  .format("com.databricks.spark.csv")
  .option("header", "true")
  .save("out/poids_par_espece_sexe")

becParIle.coalesce(1)
  .write
  .mode("overwrite")
  .format("com.databricks.spark.csv")
  .option("header", "true")
  .save("out/bec_par_ile")

evolutionAnnuelle.coalesce(1)
  .write
  .mode("overwrite")
  .format("com.databricks.spark.csv")
  .option("header", "true")
  .save("out/evolution_annuelle")

nageoiresParEspece.coalesce(1)
  .write
  .mode("overwrite")
  .format("com.databricks.spark.csv")
  .option("header", "true")
  .save("out/nageoires_par_espece")

correlationParEspece.coalesce(1)
  .write
  .mode("overwrite")
  .format("com.databricks.spark.csv")
  .option("header", "true")
  .save("out/correlation_bec_poids")

repartitionSexeIle.coalesce(1)
  .write
  .mode("overwrite")
  .format("com.databricks.spark.csv")
  .option("header", "true")
  .save("out/repartition_sexe_ile")
